# Lessson 8: The Chat Format
https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/8/chatbot

In this lesson, we will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

# Setup

In [5]:
import os

from dotenv import find_dotenv, load_dotenv
from openai import OpenAI

if not load_dotenv(find_dotenv()):
    raise RuntimeError("Set environment variables first!")

if os.getenv("OPENAI_API_KEY"):
    print("API key found")
else:
    raise RuntimeError("No API key found!")

client = OpenAI()

API key found


In [6]:
MODEL_NAME = "gpt-3.5-turbo-1106"


def get_completion(prompt: str, model: str = MODEL_NAME) -> str | None:
    """Helper function to make it easier to use prompts and look at the
    generated outpus"""
    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.0,
    )

    return response.choices[0].message.content


def get_completion_from_messages(
    messages: list[dict[str, str]], model: str = MODEL_NAME, temperature: float = 0
) -> str | None:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    return response.choices[0].message.content

In [3]:
messages = [
    {"role": "system", "content": "You are an assistant that speaks like Shakespeare."},
    {"role": "user", "content": "tell me a joke"},
    {"role": "assistant", "content": "Why did the chicken cross the road"},
    {"role": "user", "content": "I don't know"},
]

print(get_completion_from_messages(messages=messages))

To get to the other side, of course! Verily, 'tis an old jest, but one that still doth bring mirth to many.


In [4]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Hi, my name is Isa"},
]

print(get_completion_from_messages(messages=messages, temperature=1))

Hello Isa! It's nice to meet you. How can I help you today?


In [5]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Yes, can you remind me, What is my name?"},
]

print(get_completion_from_messages(messages, temperature=1))

I don't have access to personal information, including your name. But feel free to introduce yourself to me!


In [6]:
messages = [
    {"role": "system", "content": "You are friendly chatbot."},
    {"role": "user", "content": "Hi, my name is Isa"},
    {
        "role": "assistant",
        "content": "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?",
    },
    {"role": "user", "content": "Yes, you can remind me, What is my name?"},
]

print(get_completion_from_messages(messages, temperature=1))

Your name is Isa!


# OrderBot

We can automate the collection of user prompts and assistant responses to build a OrderBot. The OrderBot will take orders at a pizza restaurant.

In [7]:
import panel as pn

In [ ]:
pn.extension()

panels: list[pn.Row] = []

In [9]:
context = [
    {
        "role": "system",
        "content": """
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
""",
    }
]  # accumulate messages

In [ ]:
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter text here...")

button_conversation = pn.widgets.Button(name="Chat!")


def collect_messages(_) -> pn.Column:
    prompt = inp.value_input
    inp.value = ""
    context.append({"role": "user", "content": str(prompt)})
    response = get_completion_from_messages(context)
    context.append({"role": "assistant", "content": str(response)})
    panels.append(pn.Row("User:", pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row(
            "Assistant:",
            pn.pane.Markdown(
                response, width=600, styles={"background-color": "#F6F6F6"}
            ),
        )
    )

    return pn.Column(*panels)


interactive_conversiation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversiation, loading_indicator=True, height=300),
)

dashboard

In [14]:
messages = context.copy()
messages.append(
    {
        "role": "system",
        "content": "create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price ",
    },
)
# The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": "large pepperoni pizza - $12.95",
  "toppings": [],
  "drinks": [
    "coke - $3.00"
  ],
  "sides": [
    "fries - $4.50"
  ],
  "total price": "$20.45"
}
